In [169]:
import pandas as pd
import os
import numpy as np
from wordcloud import WordCloud
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [184]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/train_data/train.csv', header=None)
# df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)
# df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)
df = df.rename({0: 'text', 1: 'label'}, axis = 1)

In [172]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

In [173]:
df

,text,label
1,Wow... Loved this place.,1
2,Crust is not good.,0
3,Not tasty and the texture was just nasty.,0
4,Stopped by during the late May bank holiday of...,1
5,The selection on the menu was great and so wer...,1
...,...,...
996,I think food should have flavor and texture an...,0
997,Appetite instantly gone.,0
998,Overall I was not impressed and would not go b...,0
999,"The whole experience was underwhelming, and I ...",0


In [174]:
df['clean_text'] = df.text.apply(clean)
df['clean_text'] = df['clean_text'].astype('str')

In [175]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(df['clean_text'])

lda_model = LatentDirichletAllocation(n_components=4)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [176]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-5 - 1:-1]])

In [177]:
print_topics(lda_model, vectorizer)

Topic 0:
[('food', 63.17431405324073), ('good', 53.40349980476475), ('great', 42.77701187249921), ('place', 27.43256413107815), ('friendly', 27.246354006460017)]
Topic 1:
[('place', 54.46623574602995), ('service', 34.73973444795151), ('go', 31.877810293199804), ('back', 25.9231121538039), ('star', 18.24619548584007)]
Topic 2:
[('back', 30.679256679210262), ('time', 23.9372376271045), ('service', 18.8539950948999), ('place', 15.160167522936261), ('never', 15.037490507307602)]
Topic 3:
[('food', 42.33094780523769), ('good', 28.617843425717414), ('service', 17.06729557884156), ('better', 16.23645371788231), ('way', 16.19113143368664)]
